In [1]:
import warnings
warnings.filterwarnings("ignore")

import argparse
import logging
import os
import sys
import time

from collections import defaultdict, OrderedDict

import matplotlib
import numpy as np
import soundfile as sf
import torch
import yaml

from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

import sys

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from basic import plot_spectrogram

sys.path.append("../")
from pyfiles.dataset import PretrainingMelDataset
from pyfiles.utils import Dict2Obj
from pyfiles.feature_extractor import get_vocos

import seq2seq_vc
import seq2seq_vc.models
import seq2seq_vc.losses
import seq2seq_vc.trainers
import seq2seq_vc.collaters

# from seq2seq_vc.datasets import ParallelVCMelDataset
from torch.utils.data import Dataset

from seq2seq_vc.utils import read_hdf5
from seq2seq_vc.utils.types import str_or_none
# from seq2seq_vc.vocoder import Vocoder
# from seq2seq_vc.vocoder.s3prl_feat2wav import S3PRL_Feat2Wav
# from seq2seq_vc.vocoder.griffin_lim import Spectrogram2Waveform
# from seq2seq_vc.vocoder.encodec import EnCodec_decoder

# set to avoid matplotlib error in CLI environment
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt

from seq2seq_vc.schedulers.warmup_lr import WarmupLR

scheduler_classes = dict(warmuplr=WarmupLR)

class Dict2Obj(object):
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])

import joblib
import glob

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
# Dataset Variables
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"

scaler_filename = f"ckpts/scalers/LibriTTS-R.save"
scaler = joblib.load(scaler_filename)

In [3]:
size = "smaller"
conditiontype = "nocondition"

args = {}
args["rank"] = 0
# args["outdir"] = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts/pretraining_add_LibriTTS-R/"
# args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts/pretraining_{conditiontype}_LibriTTS-R/"
# args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts/pretraining_concatenation_LibriTTS-R/"
# args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts/pretraining_addition_LibriTTS-R/"
args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_melmel_smaller/"

args["config_path"] = f"./../egs/l2-arctic/cascade/conf/{size}m2mvtn.melmel.yaml"
args["init_checkpoint"] = ""
# args["resume"] = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts/pretraining_add_LibriTTS-R/checkpoint-20000steps.pkl"
args["resume"] = ""
args["distributed"] = False
args = Dict2Obj(args)

# load main config
with open(args.config_path) as f:
    config = yaml.load(f, Loader=yaml.Loader)
config.update(vars(args))

# Customization
config["model_params"]["conditiontype"] = conditiontype

In [4]:
device = torch.device("cuda")
torch.backends.cudnn.benchmark = True
torch.cuda.set_device(args.rank)
if not os.path.exists(args.outdir):
    os.makedirs(args.outdir)
    
### Dataset Preparation ###
dataset = {
    "train": PretrainingMelDataset(feat_base_dir, dataset_dir, [scaler, scaler], "train"),
    "dev": PretrainingMelDataset(feat_base_dir, dataset_dir, [scaler, scaler], "valid"),
}

collater_class = getattr(
    seq2seq_vc.collaters,
    config.get("collater_type", "ARM2MVCCollater"),
)
collater = collater_class()

sampler = {"train": None, "dev": None}
data_loader = {
    "train": DataLoader(
        dataset=dataset["train"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["train"],
        pin_memory=config["pin_memory"],
    ),
    "dev": DataLoader(
        dataset=dataset["dev"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["dev"],
        pin_memory=config["pin_memory"],
    ),
}

### Model Preparation ###
model_class = getattr(
    seq2seq_vc.models,
    config.get("model_type", "M2MVTN"),
)
model = model_class(**config["model_params"]).to(device)

if config.get("criterions", None):
    criterion = {
        criterion_class: getattr(seq2seq_vc.losses, criterion_class)(
            **criterion_paramaters
        )
        for criterion_class, criterion_paramaters in config["criterions"].items()
    }
else:
    raise ValueError("Please specify criterions in the config file.")

### optimizers and schedulers ###
optimizer_class = getattr(
    torch.optim,
    # keep compatibility
    config.get("optimizer_type", "Adam"),
)
optimizer = optimizer_class(
    model.parameters(),
    **config["optimizer_params"],
)
scheduler_class = scheduler_classes.get(config.get("scheduler_type", "warmuplr"))
scheduler = scheduler_class(
    optimizer=optimizer,
    **config["scheduler_params"],
)

### define trainer ###
trainer_class = getattr(
    seq2seq_vc.trainers,
    config.get("trainer_type", "ARM2MVCTrainer"),
)
trainer = trainer_class(
    steps=0,
    epochs=0,
    data_loader=data_loader,
    sampler=sampler,
    model=model,
    vocoder=None,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    config=config,
    device=device,
)

# load pretrained parameters from checkpoint
if len(args.init_checkpoint) != 0:
    trainer.load_trained_modules(
        args.init_checkpoint, init_mods=config["init-mods"]
    )

# resume from checkpoint
if len(args.resume) != 0:
    trainer.load_checkpoint(args.resume)

# freeze modules if necessary
if config.get("freeze-mods", None) is not None:
    assert type(config["freeze-mods"]) is list
    trainer.freeze_modules(config["freeze-mods"])

In [5]:
try:
    trainer.run()
finally:
    trainer.save_checkpoint(
        os.path.join(config["outdir"], f"checkpoint-{trainer.steps}steps.pkl")
    )
    logging.info(f"Successfully saved checkpoint @ {trainer.steps}steps.")

[train]:   0%|          | 21/300000 [00:07<18:46:51,  4.44it/s]
KeyboardInterrupt

